Load the required libraries and data. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
#now = datetime.datetime.now()

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
macro = pd.read_csv('../input/macro.csv')
train.describe()
test.describe()
# Any results you write to the current directory are saved as output.

In [ ]:
#print(test['build_year'].describe())
#print(test["life_sq"].describe())
#print(test["full_sq"].describe())
#print(test["num_room"].describe())

In [ ]:
#print(train['build_year'].describe())
#print(train["life_sq"].describe())
#print(train["full_sq"].describe())
#print(train["num_room"].describe())

In [ ]:
train.loc[train['build_year']<1692,"build_year"]=np.nan
train.loc[train['build_year']>2018,"build_year"]=np.nan
test.loc[test['build_year']<1692,"build_year"]=np.nan
test.loc[test['build_year']>2018,"build_year"]=np.nan

In [ ]:
train.loc[train['state'] == 33,"state"]=np.nan
train.loc[train['build_year'] == 20052009,"build_year"]=np.nan
train.loc[train['life_sq']>train['full_sq'],"life_sq"]=np.nan
test.loc[test['state'] == 33,"state"]=np.nan
test.loc[test['build_year'] == 20052009,"build_year"]=np.nan
test.loc[test['life_sq']>test['full_sq'],"life_sq"]=np.nan

In [ ]:
#newtrain1=train.dropna(how='any')
newtrain=train.fillna(train.mode())
y_train = newtrain["price_doc"]
x_train = newtrain.drop(["id", "timestamp", "price_doc"], axis=1)
#newtest1=test.dropna(how='any')
newtest=test.fillna(test.mode())
x_test = newtest.drop(["id", "timestamp"], axis=1)

#can't merge train with test because the kernel run for very long time

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True)        

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [ ]:
#mode
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=False)
#cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [ ]:
id_test = test.id
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

In [ ]:
output.to_csv('xgbSub.csv', index=False)